# Digital-Contact-Tracing on SocioPattern

This notebook demonstrates the use of the simulator on a SocioPattern dataset.

In [1]:
import numpy as np
import DigitalContactTracing as DCT
from load_temporal_graph import load_df_socio, build_graphs_socio, get_array_of_contacts

%load_ext autoreload
%autoreload 2

### Imports and setup

In [2]:
PARAMETERS = {"temporal_gap" : 900,                 # Temporal gap between static networks (sec)
              "memory_contacts" : 7,                # Tracing memory (days)
              "max_time_quar" : 10,                 # Quarantine duration (days)
              "policies" : {"policy1":[600]},       # Digital tracing policies [RSSI, duration (sec)]
              "eps_Is" : [0.5,0.2],                 # Isolation effectivity
              "times" : 2,                          # Number of repetition of the simulation
              "seed" : None,                        # Random seed
              "Y_i" : 10,                           # Number of initially infected people
              "nc" : 0.2,                           # Fraction of individuals who are not adopting the app
              "beta_t": 0.002,                      # Parameter defining the infectiousness probability
              "symptomatics" : 0.8,                 # Fraction of symptomatic individuals 
              "testing" : 0.25,                     # Fraction of asymptomatics who are detected via random testing
              "store":
                {"to_store": True,                  # Save the results?
                "path_to_store":"RESULTS_Socio/"}}  # Target folder to save the results

### Define the parameters of the simulation

In [3]:
df = load_df_socio("Dataset/High-School.csv")
graphs = build_graphs_socio(get_array_of_contacts(df, PARAMETERS["temporal_gap"], column_name='time'),
                            PARAMETERS["temporal_gap"])

### Load the temporal graph

In [15]:
# Initialize the random seed
np.random.seed(PARAMETERS["seed"])

# Run over each configuration
for eps_I in PARAMETERS["eps_Is"]:
    for policy in PARAMETERS["policies"]:
        filter_duration = PARAMETERS["policies"][policy][0]
        filter_rssi=None
        results = []
        
        # Print some report
        print("-" * 100)
        print('Running simulation with policy: %s' % policy)
        print('       |_ eps_I               : %+4.2d'   % eps_I)
        print('       |_ filter_duration     : %+4.2d' % filter_duration)
        print('')
        
        # Repeat the simulation for a fixed configuration 
        for i in range(PARAMETERS["times"]):
            # Print some report
            print('       Running simulation     [%4d / %4d]' % (i + 1, PARAMETERS['times']))

            # Initialize the simulation object
            dct = DCT.DigitalContactTracing(graphs = graphs,
                                            PARAMETERS = PARAMETERS,
                                            eps_I=eps_I,
                                            filter_rssi=filter_rssi,
                                            filter_duration=filter_duration,
                                            use_rssi=False)
            # Run the actual simulation
            res = dct.simulate()
            
            # Save the results    
            DCT.store_real_time(res,PARAMETERS,filter_rssi,filter_duration,eps_I)        

----------------------------------------------------------------------------------------------------
Running simulation with policy: policy1
       |_ eps_I               :  +00
       |_ filter_duration     : +600

       Running simulation     [   1 /    2]
       Running simulation     [   2 /    2]
----------------------------------------------------------------------------------------------------
Running simulation with policy: policy1
       |_ eps_I               :  +00
       |_ filter_duration     : +600

       Running simulation     [   1 /    2]
       Running simulation     [   2 /    2]


In [32]:
import cProfile
import re
cProfile.run('res = dct.simulate()', 'execution_stats')
import pstats
from pstats import SortKey
p = pstats.Stats('execution_stats')
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats(20)

Wed Jul  8 14:01:37 2020    execution_stats

         3688829 function calls in 15.463 seconds

   Ordered by: cumulative time
   List reduced from 33 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   15.463   15.463 {built-in method builtins.exec}
        1    0.000    0.000   15.463   15.463 <string>:1(<module>)
        1    0.111    0.111   15.462   15.462 DigitalContactTracing.py:232(simulate)
     1615   11.481    0.007   14.323    0.009 DigitalContactTracing.py:517(update_contacts)
   528105    1.071    0.000    2.230    0.000 DigitalContactTracing.py:370(policy)
        2    0.000    0.000    0.664    0.332 <__array_function__ internals>:2(unique)
        2    0.000    0.000    0.664    0.332 {built-in method numpy.core._multiarray_umath.implement_array_function}
        2    0.001    0.000    0.664    0.332 arraysetops.py:151(unique)
   185388    0.226    0.000    0.646    0.000 graph.py:451(__ge

In [38]:
l = [1, 2, 3]

In [39]:
l.pop(-len(l))

1

In [40]:
l

[2, 3]